Import and setup some auxiliary functions

In [ ]:
import torch
from torchvision import transforms, datasets
import numpy as np
import timeit
from collections import OrderedDict
from pprint import pformat
from torch.utils.data.sampler import *
from tqdm import tqdm
import time
from google.colab import drive
from prettytable import PrettyTable

import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

torch.multiprocessing.set_sharing_strategy('file_system')


def run(algorithm, dataset_name, filename):
    predicted_test_labels, gt_labels, run_time, parameters_count = algorithm(dataset_name)
    if predicted_test_labels is None or gt_labels is None:
      return (0, 0, 0)

    correct = 0
    total = 0
    for label, prediction in zip(gt_labels, predicted_test_labels):
      total += label.size(0)
      correct += (prediction.cpu().numpy() == label.cpu().numpy()).sum().item()   # assuming your model runs on GPU
      
    accuracy = float(correct) / total
    
    print('Accuracy of the network on the 10000 test images: %d %%' % (100 * correct / total))
    return (correct, accuracy, run_time, parameters_count)

In [ ]:
def load_data(dataset_name, device, config):
    """
    loads cifar-10 dataset using torchvision, take the last 5k of the training data to be validation data
    """
    CIFAR_training = datasets.CIFAR10(root = './data', train = True, download = True, transform = config['transforms'])
    CIFAR_test = datasets.CIFAR10(root = './data', train = False, download = True, transform = config['transforms'])
    
    training_set = torch.utils.data.Subset(CIFAR_training, list(range(0, 45000)))
    validation_set = torch.utils.data.Subset(CIFAR_training, list(range(45000, 50000)))

    train_dataloader = torch.utils.data.DataLoader(training_set, batch_size = config['batch_size'], shuffle = True)
    valid_dataloader = torch.utils.data.DataLoader(validation_set, batch_size = config['batch_size'], shuffle = True)
    test_dataloader = torch.utils.data.DataLoader(CIFAR_test ,batch_size=1, shuffle=True)

        
    return train_dataloader, valid_dataloader, test_dataloader

In [ ]:
class Net(nn.Module):
  def __init__(self):
    super(Net, self).__init__()
    self.conv1 = nn.Conv2d(3, 6, kernel_size=5)
    nn.init.xavier_uniform_(self.conv1.weight)
    self.conv2 = nn.Conv2d(6, 12, kernel_size=5)
    self.conv3 = nn.Conv2d(12, 24, kernel_size = 5)
    self.fc1 = nn.Linear(384, 64)
    self.fc2 = nn.Linear(64, 10)

  def forward(self,x):
    upsampler = nn.Upsample(scale_factor=2, mode='bilinear', align_corners=True)
    x = F.relu(self.conv1(x))
    x = F.max_pool2d(x, kernel_size=2, stride=2)
    x = upsampler(x)
    x = F.relu(self.conv2(x))
    x = F.max_pool2d(x, kernel_size=2, stride=2)
    x = F.relu(self.conv3(x))
    x = F.max_pool2d(x, kernel_size=2, stride=2)
    x = x.view(x.size(0), -1)
    x = F.sigmoid(self.fc1(x))
    x = self.fc2(x)
    return F.log_softmax(x)

In [ ]:
def train(train_dataloader, valid_dataloader, device, config):
    model = None
    train_losses = []
    train_counter = []
    validation_losses = []
    validation_counter = [i*len(train_dataloader.dataset) for i in range(config['num_epochs'] + 1)]
    log_interval = 100


    model = Net().to(device)
    optimizer = optim.SGD(model.parameters(), lr=config['lr'],
                      momentum=0.5, weight_decay=config['weight_decay'])
    
    for epoch in range(1, config['num_epochs']+1):
      for batch_idx, (data, target) in enumerate(train_dataloader):
        data = data.to(device)
        target = target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.cross_entropy(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % log_interval == 0:
          print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
            epoch, batch_idx * len(data), len(train_dataloader.dataset),
            100. * batch_idx / len(train_dataloader), loss.item()))
          train_losses.append(loss.item())
          train_counter.append(
            (batch_idx*64) + ((epoch-1)*len(train_dataloader.dataset)))
          
      model.eval()
      validation_loss = 0
      correct = 0
      with torch.no_grad():
        for data, target in valid_dataloader:
          data = data.to(device)
          target = target.to(device)
          output = model(data)
          validation_loss += F.nll_loss(output, target, size_average=False).item()
          pred = output.data.max(1, keepdim=True)[1]
          correct += pred.eq(target.data.view_as(pred)).sum()
      validation_loss /= len(valid_dataloader.dataset)
      validation_losses.append(validation_loss)
      print('\nValidation set: Avg. loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
            validation_loss, correct, len(valid_dataloader.dataset),
            100. * correct / len(valid_dataloader.dataset)))
    return model
  

In [ ]:
def save_model_colab_for_submission(model):  # if you are running on colab
  drive.mount('/content/gdrive/', force_remount=True)
  
  torch.save(model.to(torch.device("cpu")), '/content/gdrive/My Drive/model.pt') 
  

def save_model_local_for_submission(model):  # if you are running on your local machine
  torch.save(model.to(torch.device("cpu")), 'model.pt')
  
def test(model, test_dataloader, device):
  test_predictions = []
  true_labels = []
  model.eval()
  with torch.no_grad():
    for data, target in test_dataloader:
      data = data.to(device)
      target = target.to(device)
      output = model(data)
      pred = output.data.max(1, keepdim=True)[1]
      test_predictions.append(pred.cpu())
      true_labels.append(target.data.view_as(pred.cpu()))
    
  test_predictions = torch.cat(test_predictions, axis = 0)
  true_labels = torch.cat(true_labels, axis = 0)

  return test_predictions, true_labels

def count_parameters(model):
    table = PrettyTable(["Modules", "Parameters"])
    total_params = 0
    for name, parameter in model.named_parameters():
        if not parameter.requires_grad: continue
        params = parameter.numel()
        table.add_row([name, params])
        total_params+=params
    # uncomment below codes for your debugging
    print(table)
    print(f"Total Trainable Params: {total_params}")
    return total_params

def run_NN(dataset_name):
    # set parameters cifar10
  config = {
        'lr': 0.01,
        'num_epochs': 15,
        'batch_size': 20,
        'num_classes': 10,
        'regular_constant': 0,
        'weight_decay': 0.001,
        'transforms': transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))]) }
    
  device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
  
  train_dataloader, valid_dataloader, test_dataloader = load_data(dataset_name, device, config)
  
  model = train(train_dataloader, valid_dataloader, device, config)
  parameters_count = count_parameters(model)

  device = torch.device("cpu")
  start_time = timeit.default_timer()
  assert test_dataloader.batch_size == 1, 'Error: You should use use batch size = 1 for the test loader.'
  preds, labels = test(model.to(device), test_dataloader, device)
  end_time = timeit.default_timer()
  

  test_time = (end_time - start_time)
  print("Total run time of testing the model: ", test_time , " seconds.")
  
  save_model_colab_for_submission(model)
  
  return preds, labels, test_time, parameters_count

Main loop. Run time and total score will be shown below.

In [ ]:
# Don't edit this cell
def run_on_dataset(dataset_name, filename):

    correct_predict, accuracy, run_time, parameters_count = run(run_NN, dataset_name, filename)

    result = OrderedDict(
                  correct_predict=correct_predict,
                  accuracy=accuracy,
                  run_time=run_time,
                  parameters_count=parameters_count)
    return result


def main():
    filenames = { "CIFAR10": "predictions_cifar10.txt"}
    result_all = OrderedDict()
    for dataset_name in ["CIFAR10"]:
        result_all = run_on_dataset(dataset_name, filenames[dataset_name])
    with open('result.txt', 'w') as f:
        f.writelines(pformat(result_all, indent=4))
    print("\nResult:\n", pformat(result_all, indent=4))


main()

Files already downloaded and verified
Files already downloaded and verified


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:24: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


Train Epoch: 1 [0/45000 (0%)]	Loss: 2.287263
Train Epoch: 1 [2000/45000 (4%)]	Loss: 2.298413
Train Epoch: 1 [4000/45000 (9%)]	Loss: 2.315721
Train Epoch: 1 [6000/45000 (13%)]	Loss: 2.348367
Train Epoch: 1 [8000/45000 (18%)]	Loss: 2.308371
Train Epoch: 1 [10000/45000 (22%)]	Loss: 2.310966
Train Epoch: 1 [12000/45000 (27%)]	Loss: 2.323767
Train Epoch: 1 [14000/45000 (31%)]	Loss: 2.344409
Train Epoch: 1 [16000/45000 (36%)]	Loss: 2.288195
Train Epoch: 1 [18000/45000 (40%)]	Loss: 2.263333
Train Epoch: 1 [20000/45000 (44%)]	Loss: 2.186069
Train Epoch: 1 [22000/45000 (49%)]	Loss: 2.087403
Train Epoch: 1 [24000/45000 (53%)]	Loss: 2.120604
Train Epoch: 1 [26000/45000 (58%)]	Loss: 2.245763
Train Epoch: 1 [28000/45000 (62%)]	Loss: 2.184762
Train Epoch: 1 [30000/45000 (67%)]	Loss: 2.066006
Train Epoch: 1 [32000/45000 (71%)]	Loss: 1.939059
Train Epoch: 1 [34000/45000 (76%)]	Loss: 2.068529
Train Epoch: 1 [36000/45000 (80%)]	Loss: 2.040155
Train Epoch: 1 [38000/45000 (84%)]	Loss: 1.936622
Train Epoch

KeyboardInterrupt: ignored